# Air Quality System (AQS) API 

> The AQS API (https://aqs.epa.gov/aqsweb/documents/data_api.html) is the primary place to obtain row-level data from the EPA's Air Quality System (AQS) database. You must sign up (https://aqs.epa.gov/aqsweb/documents/data_api.html#signup) to obtain a key to use the API.

> Will use this jupyter notebook to develop and test the API requests. Will first develop to pull active Pb high-vol (parameter code = 14129) and PM2.5 (parameter code 88101).

> Brannon Seay, 2021-05-28

In [1]:
# Import the necessary modules
import pandas as pd
import numpy as np
import requests
import json
import os # will want to autosave data to new folder, named based on input requests
#import logging
#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline

In [2]:
# User inputs
email = 'your_email_here'
key = 'your_key_here'
parameter = '14129'
startdate = '20190101'
enddate = '20191231'
#state = '12' #have this set up to search all states. Future version could make it optional to select ind. state or do all.
state_url = 'https://aqs.epa.gov/data/api/list/states?email='+email+'&key='+key
#monitor_url = 'https://aqs.epa.gov/data/api/monitors/byState?email='+email+'&key='+key+'&param='+parameter+'&bdate='+startdate+'&edate='+enddate+'&state='+state

In [3]:
response = requests.get(state_url)
response # an output of 'Response [200]' indicates the download was successful!

<Response [200]>

In [4]:
states_dict = json.loads(response.content)

In [5]:
# Convert dictionary to DataFrame and create a list of state codes which we'll use to loop through all API urls.
df_states = pd.DataFrame.from_dict(states_dict['Data'])
state_list = df_states['code'].tolist()

In [6]:
# Loop through all states (and territories and other 'areas').
# Will likely want to exclude Mexico and Canada. What about VI and Guam? We def. want to keep Puerto Rico.
first = 0
for i in range(len(state_list)):
    monitor_url = 'https://aqs.epa.gov/data/api/monitors/byState?email='+email+'&key='+key+'&param='+parameter+'&bdate='+startdate+'&edate='+enddate+'&state='+state_list[i]
    response = requests.get(monitor_url)
    # If statement checks that response == <Response [200]>. Alternatively, could use if response.ok:, or a try/except might be best option
    if response.status_code == 200:
        dictt = json.loads(response.content)
        #print(monitor_url)
        # Since empty strings are false in Python, the below if statement will only consider the
        # url pull for a given state if it actually contains monitor 'Data'.
        if dictt['Data']:
            # if this is the first set of data, create a new dataframe. Else, concatenate.
            if first == 0:
                df_monitors = pd.DataFrame.from_dict(dictt['Data'])
            else:
                df_monitors = pd.concat([df_monitors,pd.DataFrame.from_dict(dictt['Data'])])
            first += 1
'''        else:
            print(state_list[i], "doesn't contain any Pb monitors")
    else:
        print(state_list[i], response) '''

'        else:\n            print(state_list[i], "doesn\'t contain any Pb monitors")\n    else:\n        print(state_list[i], response) '

In [7]:
print(df_monitors.shape)
df_monitors.info()

(191, 41)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 191 entries, 0 to 2
Data columns (total 41 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   state_code               191 non-null    object 
 1   county_code              191 non-null    object 
 2   site_number              191 non-null    object 
 3   parameter_code           191 non-null    object 
 4   poc                      191 non-null    int64  
 5   parameter_name           191 non-null    object 
 6   open_date                191 non-null    object 
 7   close_date               25 non-null     object 
 8   concurred_exclusions     13 non-null     object 
 9   dominant_source          128 non-null    object 
 10  measurement_scale        186 non-null    object 
 11  measurement_scale_def    186 non-null    object 
 12  monitoring_objective     191 non-null    object 
 13  last_method_code         191 non-null    object 
 14  last_method_descri

In [8]:
df_monitors.state_code.value_counts(sort=False)

40     6
51     3
18     8
12     4
01     2
47     9
48     7
13     3
26    14
17     5
42    14
72     3
20     2
29    26
19     2
45     8
39    21
27    23
22     3
06    19
04     7
21     2
Name: state_code, dtype: int64

In [9]:
df_monitors.head(10)

,state_code,county_code,site_number,parameter_code,poc,parameter_name,open_date,close_date,concurred_exclusions,dominant_source,...,address,state_name,county_name,city_name,cbsa_code,cbsa_name,csa_code,csa_name,tribal_code,tribe_name
0,01,109,0003,14129,1,Lead (TSP) LC,2009-01-01,None,None,POINT,...,HENDERSON ROAD,Alabama,Pike,Troy,45980,"Troy, AL",None,None,None,None
1,01,109,0003,14129,2,Lead (TSP) LC,2009-01-01,None,None,POINT,...,HENDERSON ROAD,Alabama,Pike,Troy,45980,"Troy, AL",None,None,None,None
0,04,007,1002,14129,2,Lead (TSP) LC,2011-01-01,2019-09-30,None,POINT,...,SR 77,Arizona,Gila,Not in a City,37740,"Payson, AZ",None,None,None,None
1,04,007,1003,14129,2,Lead (TSP) LC,2019-09-24,None,None,POINT,...,123 Hillcrest Ave,Arizona,Gila,Hayden,37740,"Payson, AZ",None,None,None,None
2,04,013,4018,14129,2,Lead (TSP) LC,2012-01-01,2019-12-31,None,MOBILE,...,1030 W. Deer Valley Rd,Arizona,Maricopa,Phoenix,38060,"Phoenix-Mesa-Scottsdale, AZ",None,None,None,None
3,04,007,1003,14129,1,Lead (TSP) LC,2016-01-01,None,None,POINT,...,123 Hillcrest Ave,Arizona,Gila,Hayden,37740,"Payson, AZ",None,None,None,None
4,04,007,8000,14129,1,Lead (TSP) LC,2011-01-01,None,None,POINT,...,SR 188 & US 60,Arizona,Gila,Globe,37740,"Payson, AZ",None,None,None,None
5,04,013,4018,14129,1,Lead (TSP) LC,2012-01-01,2019-12-31,None,MOBILE,...,1030 W. Deer Valley Rd,Arizona,Maricopa,Phoenix,38060,"Phoenix-Mesa-Scottsdale, AZ",None,None,None,None
6,04,007,1002,14129,1,Lead (TSP) LC,2011-01-01,None,None,POINT,...,SR 77,Arizona,Gila,Not in a City,37740,"Payson, AZ",None,None,None,None
0,06,037,1103,14129,3,Lead (TSP) LC,2009-01-01,None,None,None,...,"1630 N MAIN ST, LOS ANGELES",California,Los Angeles,Los Angeles,31080,"Los Angeles-Long Beach-Anaheim, CA",348,"Los Angeles-Long Beach, CA",None,None


In [10]:
df_monitors.to_csv('df_monitors.csv',index=False)